In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score
from nltk.tokenize import word_tokenize
import pandas as pd

In [10]:
df_preprocessed = pd.read_csv('C:\\Users\\Janek\\Downloads\\df_train_preprocessed.csv')
df_preprocessed

,Unnamed: 0,File,Text,Category,language
0,0,bb995ead-f65c-43e8-9d7f-771546d21651.tiff,"['parliament', 'wsrdd', 'scrambeer', 'engeyi',...",advertisement,en
1,1,2a476c78-ce1c-4535-831c-626635b69fef.tiff,"['gilvvmather', 'insertion', 'order', 'i6ken7n...",advertisement,en
2,2,7bbd6936-4f3d-4eb4-bb75-983cbca08f48.tiff,"['newport', '902', 'cosmopolitan', 'april', '1...",advertisement,en
3,3,62d852b8-25eb-48e0-8cfe-0091c4f2453e.tiff,"['goforthegołd', 'introduce', 'true', 'gqld', ...",advertisement,en
4,4,8c5a5c2b-c58d-4f7e-985d-236bcc69011e.tiff,"['what', 's', 'tobacoo', 'induisty', 'dong', '...",advertisement,en
...,...,...,...,...,...
10824,10824,3827a69b-52cb-4600-a970-5163322ac4d7.jpg,"['umowa', '0', 'dzielo', 'kukulowo', '0625', '...",umowa_o_dzielo,pl
10825,10825,671cca7a-fe43-4743-9c23-4c4a4ef5b810.jpg,"['umowa', 'odiflo', 'zawrzeć', 'dzień', '20221...",umowa_o_dzielo,pl
10826,10826,a09e8f97-35c0-4327-9ab8-dfc2b2052722.jpg,"['uowaodiflo', 'zawrzeć', 'dzień', '2020032', ...",umowa_o_dzielo,pl
10827,10827,42ede5e5-8b9c-48d7-8996-ca81e8408b90.jpg,"['umowa', '0', 'dziłlo', 'zawrzeć', 'dzień', '...",umowa_o_dzielo,pl


In [18]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
df_preprocessed['Category'] = Encoder.fit_transform(df_preprocessed['Category'])

In [52]:
df_preprocessed

,Unnamed: 0,File,Text,Category,language
0,0,bb995ead-f65c-43e8-9d7f-771546d21651.tiff,"['parliament', 'wsrdd', 'scrambeer', 'engeyi',...",0,en
1,1,2a476c78-ce1c-4535-831c-626635b69fef.tiff,"['gilvvmather', 'insertion', 'order', 'i6ken7n...",0,en
2,2,7bbd6936-4f3d-4eb4-bb75-983cbca08f48.tiff,"['newport', '902', 'cosmopolitan', 'april', '1...",0,en
3,3,62d852b8-25eb-48e0-8cfe-0091c4f2453e.tiff,"['goforthegołd', 'introduce', 'true', 'gqld', ...",0,en
4,4,8c5a5c2b-c58d-4f7e-985d-236bcc69011e.tiff,"['what', 's', 'tobacoo', 'induisty', 'dong', '...",0,en
...,...,...,...,...,...
10824,10824,3827a69b-52cb-4600-a970-5163322ac4d7.jpg,"['umowa', '0', 'dzielo', 'kukulowo', '0625', '...",19,pl
10825,10825,671cca7a-fe43-4743-9c23-4c4a4ef5b810.jpg,"['umowa', 'odiflo', 'zawrzeć', 'dzień', '20221...",19,pl
10826,10826,a09e8f97-35c0-4327-9ab8-dfc2b2052722.jpg,"['uowaodiflo', 'zawrzeć', 'dzień', '2020032', ...",19,pl
10827,10827,42ede5e5-8b9c-48d7-8996-ca81e8408b90.jpg,"['umowa', '0', 'dziłlo', 'zawrzeć', 'dzień', '...",19,pl


In [53]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_preprocessed, test_size=0.2, random_state=42)

In [54]:
X_train = df_train['Text']
y_train = df_train['Category']
X_val = df_val['Text']
y_val = df_val['Category']
X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_train_tokens)
Train_X_Tfidf = Tfidf_vect.transform(X_train)

In [44]:
print(Train_X_Tfidf)

  (0, 4615)	0.18352867580570673
  (0, 2796)	0.17798074681899534
  (0, 2519)	0.15072423989443579
  (0, 2184)	0.1372953394983992
  (0, 1995)	0.15734900442664815
  (0, 1508)	0.21413471346519147
  (0, 1476)	0.19124094441881725
  (0, 483)	0.2093073046724029
  (0, 463)	0.1974358591286235
  (0, 445)	0.18234019164364523
  (0, 438)	0.41452356331234413
  (0, 433)	0.14474102107160766
  (0, 432)	0.44061940212279105
  (0, 407)	0.18352867580570673
  (0, 335)	0.15752885729896207
  (0, 180)	0.15844346541029555
  (0, 179)	0.3494994035515548
  (0, 107)	0.2430504387182748
  (0, 0)	0.13452648570070488
  (1, 4994)	0.21611645064585033
  (1, 4951)	0.14630838366744145
  (1, 4874)	0.18976824974109396
  (1, 4740)	0.23681971169439403
  (1, 4605)	0.17779961948913106
  (1, 4178)	0.2718512299539105
  :	:
  (8662, 2664)	0.12616458164366204
  (8662, 2585)	0.10243288688806566
  (8662, 2584)	0.10415866853613279
  (8662, 2467)	0.08556737043008432
  (8662, 2338)	0.08857669339432378
  (8662, 1939)	0.08643116588405926
  (8

## Naive Bayes

In [28]:
def training_pipeline_bayes(df_train, df_val):
    X_train = df_train['Text']
    y_train = df_train['Category']
    X_val = df_val['Text']
    y_val = df_val['Category']
    X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Val_X_Tfidf = Tfidf_vect.transform(X_val)
    
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf, y_train)
    predictions_NB = Naive.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_NB, y_val)
    f1 = f1_score(predictions_NB, y_val, average='macro')
    print("Naive Bayes Accuracy Score:", accuracy * 100)
    print("Naive Bayes F1 Score:", f1 * 100)
    return predictions_NB

In [27]:
training_pipeline_bayes(df_train, df_val)

Naive Bayes Accuracy Score: 67.12834718374884
Naive Bayes F1 Score: 68.75839993855612


array(['invoice', 'handwritten', 'handwritten', ..., 'resume', 'pit37_v1',
       'handwritten'], dtype='<U42')

## SVM

In [56]:
def training_pipeline_svm(df_train, df_val):
    X_train = df_train['Text']
    y_train = df_train['Category']
    X_val = df_val['Text']
    y_val = df_val['Category']
    X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Val_X_Tfidf = Tfidf_vect.transform(X_val)
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf,y_train)
    predictions_SVM = SVM.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_SVM, y_val)
    f1 = f1_score(predictions_SVM, y_val, average='macro')
    print("SVM Accuracy Score:", accuracy * 100)
    print("SVM:", f1 * 100)
    return predictions_SVM

In [57]:
training_pipeline_svm(df_train, df_val)

SVM Accuracy Score: 72.80701754385966
SVM: 75.45862478683715


array([ 6,  3,  3, ..., 16, 10,  5], dtype=int64)

## XGBoost

In [39]:
import xgboost as xgb

def training_pipeline_xgb(df_train, df_val):
    X_train = df_train['Text']
    y_train = df_train['Category']
    X_val = df_val['Text']
    y_val = df_val['Category']
    
    X_train_tokens = [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Val_X_Tfidf = Tfidf_vect.transform(X_val)
    
    xgb_model = xgb.XGBClassifier()
    xgb_model.fit(Train_X_Tfidf, y_train)
    
    predictions_xgb = xgb_model.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_xgb, y_val)
    f1 = f1_score(predictions_xgb, y_val, average='macro')
    
    print("XGBoost Accuracy Score:", accuracy * 100)
    print("XGBoost F1 Score:", f1 * 100)
    
    return predictions_xgb

In [40]:
training_pipeline_xgb(df_train, df_val)

C:\Users\Janek\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:14:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Accuracy Score: 69.71375807940905
XGBoost F1 Score: 72.44792115047309


array(['budget', 'file_folder', 'file_folder', ..., 'file_folder',
       'pit37_v1', 'handwritten'], dtype=object)

## LinearSVC

In [45]:
from sklearn.svm import LinearSVC

def training_pipeline_linear(df_train, df_val):
    X_train = df_train['Text']
    y_train = df_train['Category']
    X_val = df_val['Text']
    y_val = df_val['Category']
    
    X_train_tokens = [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Val_X_Tfidf = Tfidf_vect.transform(X_val)
    
    linear_model = LinearSVC()
    linear_model.fit(Train_X_Tfidf, y_train)
    
    predictions_linear = linear_model.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_linear, y_val)
    f1 = f1_score(predictions_linear, y_val, average='macro')
    
    print("Linear Classifier Accuracy Score:", accuracy * 100)
    print("Linear Classifier F1 Score:", f1 * 100)
    
    return predictions_linear

In [46]:
training_pipeline_linear(df_train, df_val)

Linear Classifier Accuracy Score: 71.37580794090489
Linear Classifier F1 Score: 73.74887330222634


array(['invoice', 'file_folder', 'file_folder', ..., 'resume', 'pit37_v1',
       'invoice'], dtype=object)

## Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier

def training_pipeline_rf(df_train, df_val):
    X_train = df_train['Text']
    y_train = df_train['Category']
    X_val = df_val['Text']
    y_val = df_val['Category']
    
    X_train_tokens = [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Val_X_Tfidf = Tfidf_vect.transform(X_val)
    
    rf_model = RandomForestClassifier()
    rf_model.fit(Train_X_Tfidf, y_train)
    
    predictions_rf = rf_model.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_rf, y_val)
    f1 = f1_score(predictions_rf, y_val, average='macro')
    
    print("Random Forest Accuracy Score:", accuracy * 100)
    print("Random Forest F1 Score:", f1 * 100)
    
    return predictions_rf

In [48]:
training_pipeline_rf(df_train, df_val)

Random Forest Accuracy Score: 69.80609418282549
Random Forest F1 Score: 72.44596602375542


array(['form', 'file_folder', 'file_folder', ..., 'resume', 'pit37_v1',
       'budget'], dtype=object)